In [1]:
import os
os.getcwd()

'D:\\Yifeng -- Project Work\\ys8mz_sandbox\\degree_completion_s1'

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import confusion_matrix

In [3]:
df = pd.read_stata("pred_score_by_race.dta")

In [4]:
def two_side_z_test(n1, p1, n2, p2):
    z = (p2-p1)/np.sqrt(p2*(1-p2)/(n2-1)+p1*(1-p1)/(n1-1))
    return 2*(1-stats.norm.cdf(np.abs(z)))

In [5]:
new_pred_real = pd.DataFrame({'pred_y': list(df.pred_y),'real_y': list(df.test_y), 'race': list(df.race_column)}).sample(frac=1, random_state=54321)
# new_pred_real.loc[:,'pred_y_binned'] = pd.cut(new_pred_real.pred_y, bins=[0] + list(np.percentile(new_pred_real.pred_y, np.arange(2,100,2))) + [1])
# new_pred_real.loc[:,'pred_y_binned_2'] = pd.cut(new_pred_real.pred_y, bins=[0] + list(np.percentile(new_pred_real.pred_y, np.arange(10,100,10))) + [1])

In [6]:
white_df = new_pred_real[new_pred_real.race == "white"]
white_df = white_df.iloc[np.argsort(white_df.pred_y), :]
afam_df = new_pred_real[new_pred_real.race == "afam"]
afam_df = afam_df.iloc[np.argsort(afam_df.pred_y), :]

In [7]:
white_df.shape[0], afam_df.shape[0]

(24934, 12007)

In [8]:
results = []
for qq in [0.1, 0.2, 0.25, 0.3, 0.33, 0.4, 0.5, 0.6, 0.67, 0.7, 0.75, 0.8, 0.9]:
    print("qq = {}".format(qq))
    q = np.quantile(new_pred_real.pred_y, qq)
    new_pred_real_pct = new_pred_real[new_pred_real.pred_y < q]
    m1 = new_pred_real_pct.shape[0]
    m2 = int(new_pred_real.shape[0] * qq)
    if m1 < m2:
        new_pred_real_pct2 = new_pred_real[new_pred_real.pred_y >= q]
        new_pred_real_pct2 = new_pred_real_pct2.iloc[np.argsort(new_pred_real_pct2.pred_y), :]
        new_pred_real_pct = pd.concat([new_pred_real_pct, new_pred_real_pct2.iloc[:(m2-m1), :]])
    elif m1 > m2:
        new_pred_real_pct = new_pred_real_pct.iloc[:m2, :]
    new_pred_real_pct = new_pred_real_pct[new_pred_real_pct.race.apply(lambda x: x in {'white', 'afam'})]
    new_pred_real_pct_agg = new_pred_real_pct.groupby(['race']).agg({'real_y': 'mean', 'pred_y': 'count'}).sort_index().loc[:,['real_y', 'pred_y']]
    n1, p1 = new_pred_real_pct_agg.iloc[0,1], new_pred_real_pct_agg.iloc[0,0]
    n2, p2 = new_pred_real_pct_agg.iloc[1,1], new_pred_real_pct_agg.iloc[1,0]
    p_value_1 = two_side_z_test(n1, p1, n2, p2)
    ratio_1 = n1 / n2
    pp1, pp2 = 0, 1
    j = 1
    # while p_value < 0.05:
    while pp1 < pp2 and j < n2 and n1 + j <= afam_df.shape[0]:
        j += 1
        white_df_new = white_df.iloc[0:(n2-j),:]
        afam_df_new = afam_df.iloc[0:(n1+j),:]
        merged_new = pd.concat([white_df_new, afam_df_new])
        merged_new_agg = merged_new.groupby(['race']).agg({'real_y': 'mean', 'pred_y': 'count'}).sort_index().loc[:,['real_y', 'pred_y']]
        nn1, pp1 = merged_new_agg.iloc[0,1], merged_new_agg.iloc[0,0]
        nn2, pp2 = merged_new_agg.iloc[1,1], merged_new_agg.iloc[1,0]
    p_value_2 = two_side_z_test(nn1, pp1, nn2, pp2)
    ratio_2 = nn1/nn2
    results.append((int(qq*100), p1, p2, n1, n2, pp1, pp2, nn1, nn2, nn1 - n1))

qq = 0.1
qq = 0.2
qq = 0.25
qq = 0.3
qq = 0.33
qq = 0.4
qq = 0.5
qq = 0.6
qq = 0.67
qq = 0.7
qq = 0.75
qq = 0.8
qq = 0.9


In [9]:
results_df = \
pd.DataFrame(results, columns=["bottom_%", "success_rate_black", "success_rate_white", "n1", "n2",
                               "success_rate_black", "success_rate_white", "nn1","nn2",
                               "black_increased"]).sort_values(["bottom_%"]).round(4)
results_df.to_csv("calibration_bias.csv", index=False)